In [ ]:
import tensorflow as tf
import cv2
import os
import numpy as np


: 

In [ ]:
root_path = os.getcwd()
train_path = root_path+"/cats_and_dogs_filtered/train"
val_path = root_path+"/cats_and_dogs_filtered/validation"

In [ ]:
input_size = (64,64)
input_shape = (64,64,3)

def load_img(img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(img,input_size)
    return img


In [ ]:

X_train,Y_train = [],[]
X_val,Y_val = [],[]


# Load Train
for cur_class in os.listdir(train_path):
    label = ""
    if cur_class == "cats":
        label = [1,0]
    else:
        label = [0,1]

    for filename in os.listdir(train_path+"/"+cur_class):
        img_path = train_path+"/"+cur_class+"/"+filename
        img = load_img(img_path)
        X_train.append(img)
        Y_train.append(label)



# Load Test
for cur_class in os.listdir(val_path):
    label = ""
    if cur_class == "cats":
        label = [1,0]
    else:
        label = [0,1]

    for filename in os.listdir(val_path+"/"+cur_class):
        img_path = val_path+"/"+cur_class+"/"+filename
        img = load_img(img_path)
        X_val.append(img)
        Y_val.append(label)


X_train = np.array(X_train)
Y_train = np.array(Y_train)
X_val = np.array(X_val)
Y_val = np.array(Y_val)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

index = 19

figure(figsize=(8, 6), dpi=80)
plt.subplot(1,2,1)
plt.title("Train Dog")
plt.imshow(X_train[index])


plt.subplot(1,2,2)
plt.title("Val Dog")
plt.imshow(X_val[index])



plt.show()

In [ ]:
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,Dense,Flatten,MaxPooling2D,Dropout


model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),  # Conv Layer 1
    MaxPooling2D((2, 2)),                                           # Max Pooling Layer 1
    Conv2D(64, (3, 3), activation='relu'),                          # Conv Layer 2
    MaxPooling2D((2, 2)),                                           # Max Pooling Layer 2
    Flatten(),                                                      # Flatten Layer
    Dense(128, activation='relu'),                                  # Fully Connected Layer
    Dense(2, activation='softmax')      
])
model.summary()

#### 1. Conv Layer 1:
- Input Shape: (64, 64, 3)
- Number of Filters: 32
- Kernel Size: (3, 3)
- Parameters: (3 * 3 * 3 + 1) * 32 = 896


#### 3. Conv Layer 2:
- Input Shape: (31, 31, 32) (The size is halved after max pooling)
- Number of Filters: 64
- Kernel Size: (3, 3)
- Parameters: (3 * 3 * 32 + 1) * 64 = 18,496



#### 5. Flatten Layer:
- The output from the second conv layer and max pooling is reshaped into a flat vector.
- Shape after second pooling: (14, 14, 64)
- Total Units after Flatten: 14 * 14 * 64 = 12,544
- No trainable parameters here, as this layer just reshapes the data.

#### 6. Fully Connected (Dense) Layer:
- Input Size: 12,544
- Output Neurons: 128
- Parameters: (12,544 + 1) * 128 = 1,605,760

#### 7. Output Layer:
- Input Size: 128
- Output Neurons: 2 (for binary classification)
- Parameters: (128 + 1) * 2 = 258



- Conv Layer 1: 896
- Conv Layer 2: 18,496
- Dense Layer: 1,605,760
- Output Layer: 258
- Total Trainable Parameters = 896 + 18,496 + 1,605,760 + 258 = 1,625,410

In [ ]:

tensorflow.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
from tensorflow.keras.optimizers import Adam

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(0.001),
              metrics=['accuracy'])

In [ ]:
history = model.fit(
    x = np.array(X_train),
    y = np.array(Y_train),
    validation_data = (X_val,Y_val),
    batch_size=16,
    epochs=10,
    verbose=1)


In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')

plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
labels = ["cat","dog"]

test_img_path = "./cats_and_dogs_filtered/validation/cats/cat.2000.jpg"



test_img = np.array([load_img(test_img_path)])
print(test_img.shape)

pred_ = model.predict(test_img)
pred_label = labels[np.argmax(pred_)]
print(pred_label)
plt.imshow(test_img[0])

In [ ]:
eval_loss, eval_acc = model.evaluate(X_val,Y_val)
print('Eval loss: {}, Eval accuracy: {}'.format(eval_loss, eval_acc))

In [ ]:
#HDF 5 save format 
model_file = "sample_model.h5"
model.save(model_file)


In [ ]:
new_model = tf.keras.models.load_model(model_file)
eval_loss, eval_acc = model.evaluate(X_val,Y_val)
print('Eval loss: {}, Eval accuracy: {}'.format(eval_loss, eval_acc))
new_model.summary()